# Script untuk Download Data GDELT sampai ke Dataframe
Script ini bersumber dari <a href=http://nbviewer.jupyter.org/github/JamesPHoughton/Published_Blog_Scripts/blob/master/GDELT%20Wrangler%20-%20Clean.ipynb>James P. Houghton</a> dengan sedikit modifikasi pada beberapa bagian.

# 1. Identifikasi File yang akan kita download

In [ ]:
# impor modul yang dibutuhkan
import requests
import lxml.html as lh
import os

# link yang berisi list url file yang akan di download
gdelt_base_url = 'http://data.gdeltproject.org/events/'

# periode data yang akan di download, dapat diganti sesuai kebutuhan
begin_date = '20181001'
end_date = '20181031'

if begin_date < '200600':
    begin_date = begin_date[0:4]
elif begin_date < '20130401':
    begin_date = begin_date[0:6]

if end_date < '200600':
    end_date = end_date[0:4]+'.zip'
    
print(begin_date)
print(end_date)

# mengambil list seluruh link yang ada pada halaman file gdelt
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# menyeleksi/memfilter link sesuai dengan periode yang kita inginkan
file_list = [x for x in link_list if str.isdigit(x[0:4])]
file_list = [y for y in file_list if end_date >= str(y[0:8]) >= begin_date]

# sebagai kontrol untuk link dan periode yang telah kita tentukan
print(file_list)

# 2. Mengekstrak baris pada GDELT ke file perantara (intermediate files)

<p>Di tahap ini, kita memfilter baris data yang akan kita ekstrak berdasarkan kode negara tertentu, dan membuat file perantara baru yang merupakan copy dari file GDELT (input file) yang didownload. File perantara tersebut memiliki ukuran yang <b>lebih kecil</b> dari file input nya karena telah mengalami penyeleksian berdasarkan filter kode negara yang kita tentukan.</p>

Kode negara yang merupakan parameter yang akan kita syaratkan pada row yang akan di ekstrak didasarkan pada 3 kolom yaitu:
1. ActionGeo_CountryCode (51)
2. Actor1Geo_CountryCode (37)
3. Actor2Geo_CountryCode (44)
Daftar kode negara dapat dilihat pada link berikut, <a href=http://en.wikipedia.org/wiki/List_of_FIPS_country_codes>FIPS Country Code</a>.

In [2]:
infilecounter = 0
outfilecounter = 0

Secara umum, algoritma yang akan kita gunakan adalah sebagai berikut:

- Mendapatkan list file gdelt yang akan di download
- Dari setiap elemen pada list
> - Cek apakah file tersebut telah ada di direktori lokal kita
>> - Jika belum, script akan mendownloadnya
>> - Jika sudah, script akan langsung ke proses selanjutnya yaitu:
> - Unzip file gdelt
> - Menghasilkan masing-masing satu file CSV pada folder /tmp
>> - Membuat dan membuka file output (TSV)
>> - Dari setiap baris input file (CSV):
>>> - Membaca string yang ada
>>> - Memisahkan berdasarkan delimiternya yaitu 'tab'
>>> - Cek parameter (Kode Negara) pada kolom yang ditentukan:
>>>> - Jika tidak memenuhi, lanjutkan looping
>>>> - Jika memenuhi, tulis baris tersebut pada output (TSV) file
>> - Menambahkan hitungan jumlah file yang telah diproses
>> - Menghapus input (CSV) file

In [ ]:
import os.path
import urllib
import zipfile
import glob
import operator

local_path = '/Users/bram/Documents/Digital Talent Scholarship 2018/GDELT_Project/GDELT_Data/'
os.makedirs(os.path.dirname(local_path), exist_ok=True)

fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print (compressed_file),
    
    # Jika kita tidak memiliki file nya di direktori lokal, script akan mendownloadnya
    while not os.path.isfile(local_path+compressed_file): 
        print ('downloading,'),
        urllib.request.urlretrieve(url=gdelt_base_url+compressed_file, 
                           filename=local_path+compressed_file)
        
    else:    
        # Mengekstrak zip file ke folder sementara    
        print ('extracting,'),
        z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
        z.extractall(path=local_path+'tmp/')
    
    # Menguraikan tiap file csv ke working directory, 
    print ('parsing,'),
    for infile_name in glob.glob(local_path+'tmp/*'):
        if fips_country_code == '':
            outfile_name = local_path+'country/'+'%04i.tsv'%outfilecounter
            os.makedirs(os.path.dirname(outfile_name), exist_ok=True)
            
        else:
            outfile_name = local_path+'country/'+fips_country_code+'%04i.tsv'%outfilecounter
            os.makedirs(os.path.dirname(outfile_name), exist_ok=True)
        
        # membuka infile dan outfile
        with open(infile_name, mode='r', encoding="utf-8") as infile, open(outfile_name, mode='w', encoding="utf-8") as outfile:
            for line in infile:
                # mengekstrak baris sesuai dengan kode negara yang telah ditentukan
                if fips_country_code == '':
                    outfile.write(line)
                    
                else:
                    if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                        outfile.write(line)
            outfilecounter +=1
            
        # menghapus file sementara
        os.remove(infile_name)
    infilecounter +=1
    print ('done')

# 3. Membangun file perantara ke Pandas Dataframe
<p>Algoritma yang digunakan cukup sederhana yaitu mebangun dataframe dari setiap file sementara yang dihasilkan dari proses sebelumnya (TSV) kemudian menyatukannya ke dalam satu dataframe yang besar, menyimpan dataframe tersebut lalu menghapus file sementara yang sudah tidak dibutuhkan.</p>
<p>Untuk memberikan header pada dataframe kita, digunakan file pada link berikut:</p>
<br><a href=http://gdeltproject.org/data/lookups/CSV.header.fieldids.xlsx>http://gdeltproject.org/data/lookups/CSV.header.fieldids.xlsx</a>

In [ ]:
import glob
import pandas as pd

# Mengambil nama kolom file GDELT dari file 
colnames = pd.read_excel(local_path+'CSV.header.fieldids.xlsx', sheet_name='Sheet1', 
                         index_col='Column ID', usecols=1)['Field Name']

# Membangun dataframe dari setiap file perantara (TSV)
if fips_country_code == '':
    files = glob.glob(local_path+'country/'+'*')    
    
else:
    files = glob.glob(local_path+'country/'+fips_country_code+'*')
    
DFlist = []
for active_file in files:
    print (active_file)
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID']))

# Menggabungkan masing-masing dataframe dari proses sebelumnya, dan membuat file backup
print ("merging the file-based dataframes and save a pickle backup...tunggu..."),
DF = pd.concat(DFlist)
if fips_country_code == '':
    DF.to_pickle(local_path+'backup'+'.pickle')  
    
else:
    DF.to_pickle(local_path+'backup'+fips_country_code+'.pickle')     
    
# Menghapus file sementara
for active_file in files:
    os.remove(active_file)

# 4. Mengekspor dataframe ke database MySQL

### Setting Koneksi

In [ ]:
import pymysql #mengimport module pymsql
import pandas as pd #mengimport module pandas
from sqlalchemy import create_engine #import modul create_engine dari library sqlalchemy

#settingan database
host = '127.0.0.1'   #ip database
port = "3306"        #port database
username = 'root'    #username
password = ''        #password
database = 'gdeltproject' #nama database

#deklarasi koneksi ke database
engine = create_engine('mysql+pymysql://'+username+':'+password+'@'+host+':'+port+'/'+database)
# engine = create_engine('mysql+pymysql://root@localhost:3306/classicmodels')

#deklarasi fungsi run
def run(sql):
    query = pd.read_sql_query(sql, engine)
    return query

### Proses ekspor

In [ ]:
DF.to_sql(name= 'events', con = engine, if_exists = 'replace', index = True)

### Catatan:
Mohon dicek jika ada bug dan semacamnya..
<br>Sudah saya coba sampai proses ekspor ke database MySQL berjalan lancar, dengan catatan data tidak terlalu banyak..
<br>Saya tes pake data events GDELT bulan Oktober 2018 dengan kode negara ID, file backupID.pickle yang dihasilkan hanya sekitar 16 MB, sampai ke MySQL, lancar jaya...
<br>Tapi ketika data tidak difilter, file backup.pickle yang dihasilkan sekitar 2 GB, laptop saya (I7-8750H, RAM 8 GB) nggak sanggup....